# Packages

In [8]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from numpy.random import randint, uniform
import time
# from pykdtree.kdtree import KDTree
from collections import defaultdict, Counter

# Wasserstein Distance

In [74]:
a = np.array([randint(low = 0, high = 1000, size = 1) for i in range(100)])
b = np.array([randint(low = 0, high = 1000, size = 1) for i in range(1)])
c = np.ravel(np.array([uniform(low = 0, high = 1, size = 100) for i in range(1)]))

In [64]:
p = 2

In [66]:
%timeit 45849584858**(1/p)

6.22 ns ± 0.00327 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [55]:
%timeit np.sum(c * np.sum(np.abs(a - b)**p, axis = 1))**(1/p)

8.44 µs ± 366 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [38]:
np.linalg.norm(x = a - b, axis = 1, ord = 2)

array([659., 819.,  10., 208., 532.,  34., 235., 322., 417., 436.,  31.,
       836., 638., 839., 234., 217., 349., 608., 149.,  87., 477., 629.,
       322.,   1., 128.,  37., 167., 261., 295., 162., 699., 233., 105.,
       414., 619., 253., 300., 109., 726.,  54., 441.,  87.,  14.,  22.,
       174.,  99., 553., 695., 164.,  92., 575., 862.,  70., 459., 358.,
       625., 442., 423.,  55., 760.,  63., 443., 481., 799., 487.,  12.,
        89., 467.,  87., 485., 363.,  83., 521., 631.,   3., 123., 467.,
       711., 153.,  35.,   1., 806.,  69., 743., 732., 666., 814., 740.,
       343., 443., 244., 651.,  71., 491., 324., 228., 775., 707., 430.,
       150.])

In [33]:
a = np.array([[randint(low = 0, high = 10, size = 100) for i in range(4)] for j in range(10)])
p = np.array([[uniform(low = 0, high = 1, size = 100) for i in range(1)] for j in range(10)])

b = np.array([[randint(low = 0, high = 10, size = 100) for i in range(4)] for j in range(1)])

In [34]:
a.shape

(10, 4, 100)

In [35]:
b.shape

(1, 4, 100)

In [36]:
p.shape

(10, 1, 100)

In [42]:
res = list()

for i in range(a.shape[2]):
    aSlice = a[:, :, i]
    pSlice = p[:, :, i]
    bSlice = b[:, :, i]
    
    wasserstein = np.sum(pSlice * np.sum(np.abs(aSlice - bSlice)**2, axis = 1))**(1/2)
    res.append(wasserstein)

In [51]:
test = np.abs(a - b)**2

In [52]:
test.shape

(10, 4, 100)

In [56]:
test2 = np.sum(test, axis = 1)

In [62]:
pMod = np.squeeze(p, axis = 1)

In [63]:
test3 = pMod * test2

In [69]:
test4 = np.sum(test3, axis = 0)**(1/2)

In [70]:
test4

array([14.90681762, 17.58025152, 14.09993819, 14.19721947, 20.42119541,
       21.17083643, 21.82506519, 14.28860965, 17.95907864, 17.63935027,
       19.02613227, 20.36836659, 20.29427056, 22.73503271, 10.89135527,
       15.27272374, 19.22067102, 20.87057817, 12.64432795, 16.34983916,
       19.13416765, 18.39450796, 14.02995581, 14.3509082 , 11.92153673,
       21.02701729, 13.36082062, 22.18488438, 18.45995844, 17.9084535 ,
       21.45640724, 14.29555104, 15.58426005, 17.3009952 , 13.14802228,
       17.86225696, 16.74098367, 17.65829595, 21.99450171, 17.63190717,
       17.26877606, 13.92378275, 14.88482145, 22.55893115, 15.48559782,
       21.2163428 , 17.98669987, 15.87471136, 16.75928126, 14.2697816 ,
       18.44787782, 14.53851229, 16.46989668, 18.2880547 , 17.50195144,
       19.89996158, 17.58243829, 14.47188704, 20.38860254, 24.00839087,
       12.75877816, 16.61880167, 19.43927195, 13.66638184, 19.7897643 ,
       16.60904455, 14.95741948, 20.73879401, 20.7678843 , 20.89

## One dimensional improvement

In [38]:
dimY = 1
nTest = 5000
maxPosProb = 100

densities = list()
yTest = list()

for j in range(nTest):
    nPosProb = int(randint(low = 1, high = maxPosProb, size = 1))
    
    probs = np.ravel([uniform(low = 0, high = 1, size = 1) for i in range(nPosProb)])
    probs = probs / probs.sum()
    
    values = np.ravel(np.array([randint(low = 0, high = 10, size = dimY) for i in range(nPosProb)]))
    
    densities.append((probs, values))
    yTest.append(randint(low = 1, high = 10, size = dimY))

yTest = np.array(yTest)

In [39]:
%%timeit

lenList = list()
valuesList = list()
probsList = list()

for probs, values in densities:
    lenList.append(values.shape[0])
    valuesList.append(values)
    probsList.append(probs)
    
lenMax = max(lenList)

insertionCheck = np.arange(lenMax) < np.array(lenList)[:, None]

probsArray = np.zeros(shape = (len(probsList), lenMax))
valuesArray = np.zeros(shape = (len(valuesList), lenMax))

probsArray[insertionCheck] = np.concatenate(probsList, axis = 0)
valuesArray[insertionCheck] = np.concatenate(valuesList, axis = 0)

# yTest = np.concatenate(yList, axis = 0).reshape(-1, 1)

distances = np.sum(probsArray * np.abs(valuesArray - yTest)**2, axis = 1)**(1/2)

11.4 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [40]:
%%timeit 

# BENCHMARK

wassersteinDists = list()
    
for i in range(len(densities)):
    y = yTest[i]
    values = densities[i][1]
    probs = densities[i][0]
    
    wassersteinDists.append(np.sum(probs * np.abs(values - y)**2)**(1/2))
    
wassersteinDists = np.array(wassersteinDists)

52.8 ms ± 2.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Multi-dimensional improvement

In [297]:
from scipy.spatial.distance import cdist

dimY = 4
nTest = 5000
maxPosProb = 500

densities = list()
yTest = list()

for j in range(nTest):
    nPosProb = int(randint(low = 1, high = maxPosProb, size = 1))
    
    probs = np.ravel([uniform(low = 0, high = 1, size = 1) for i in range(nPosProb)])
    probs = probs / probs.sum()
    
    values = np.array([randint(low = 0, high = 10, size = dimY) for i in range(nPosProb)])
    
    densities.append((probs, values))
    yTest.append(randint(low = 1, high = 10, size = dimY))

yTest = np.array(yTest)

In [313]:
%%timeit

# BENCHMARK

wassersteinDists = list()
    
for i in range(len(densities)):
    y = yTest[i]
    values = densities[i][1]
    probs = densities[i][0]
    
    wassersteinDists.append(np.sum(probs * np.sum(np.abs(values - y)**2, axis = 1))**(1/2))
    
wassersteinDists = np.array(wassersteinDists)

146 ms ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [311]:
%%timeit

yDim = densities[0][1].shape[1]

lenList = list()
for probs, values in densities:
    lenList.append(values.shape[0])
    
lenMax = max(lenList)
valuesArray = np.zeros(shape = (len(densities), yDim, lenMax))
probsArray = np.zeros(shape = (len(densities), lenMax))

for i in range(len(densities)):
    length = lenList[i]
    values = densities[i][1]
    probs = densities[i][0]
    
    valuesArray[i, :, 0:length] = values.T
    probsArray[i, 0:length] = probs

yTestMod = np.expand_dims(a = yTest, axis = 2)

distances = np.sum(np.abs(valuesArray - yTestMod)**2, axis = 1)

wassersteinDists2 = np.sum(probsArray * distances, axis = 1)**(1/2)

142 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [310]:
%%timeit

yDim = densities[0][1].shape[1]

lenList = list()
probsList = list()
valuesListDict = defaultdict(list)

for probs, values in densities:
    lenList.append(values.shape[0])
    probsList.append(probs)
    
    for i in range(yDim):
        valuesListDict[i].append(values[:, i])
    
lenMax = max(lenList)
insertionCheck = np.arange(lenMax) < np.array(lenList)[:, None]

#---

valuesArrayList = list()

for i in range(yDim):
    valuesArray = np.zeros(shape = (len(densities), lenMax))
    valuesArray[insertionCheck] = np.concatenate(valuesListDict[i], axis = 0)
    
    valuesArray = np.expand_dims(valuesArray, axis = 2)
    
    valuesArrayList.append(valuesArray)

valuesArray = np.concatenate(valuesArrayList, axis = 2)
valuesArray = np.swapaxes(valuesArray, axis1 = 1, axis2 = 2)

probsArray = np.zeros(shape = (len(densities), lenMax))
probsArray[insertionCheck] = np.concatenate(probsList, axis = 0)
        
yTestMod = np.expand_dims(a = yTest, axis = 2)

#---

distances = np.sum(np.abs(valuesArray - yTestMod)**2, axis = 1)
wassersteinDists3 = np.sum(probsArray * distances, axis = 1)**(1/2)

310 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [236]:
test = np.repeat(a = np.arange(lenMax).reshape(1, -1), repeats = nTest, axis = 0)
test2 = np.expand_dims(test, axis = 2)

test3 = np.repeat(a = test2, repeats = yDim, axis = 2)
test4 = np.swapaxes(test3, axis1 = 1, axis2 = 2)

comp = np.repeat(a = np.array(lenList).reshape(-1, 1), repeats = yDim, axis = 1)
comp2 = np.expand_dims(comp, axis = 2)

test5 = test4 < comp2

valuesList = list()
for probs, values in densities:
     valuesList.append(values)
        
valuesArray = np.zeros(shape = (len(densities), yDim, lenMax))

# valuesArray[test5] = np.concatenate(valuesList)

In [238]:
np.arange(lenMax) < 

9

In [243]:
test = np.arange(lenMax) < np.array(lenList)[:, None]


array([ True,  True,  True,  True, False, False, False, False, False])

TypeError: NumPy boolean array indexing assignment requires a 0 or 1-dimensional input, input has 2 dimensions

In [230]:
test5.shape

(100, 4, 9)

In [221]:
indices = np.array([[[1, 2, 3], [2, 3, 4]]])

In [223]:
valuesArray[indices].shape

(1, 2, 3, 4, 9)

In [ ]:
valuesList = [np.ravel([randint(low = 0, high = 10, size = 1) for i in range(int(randint(low = 1, high = 10, size = 1)))]) for j in range(100)]
probsList = [np.ravel(np.array([uniform(low = 0, high = 1, size = 1) for i in range(valuesList[j].shape[0])])) for j in range(100)]
probsList = [probs / probs.sum() for probs in probsList]

yList = [np.ravel([randint(low = 0, high = 10, size = 1) for i in range(1)]) for j in range(100)]

In [49]:
test = np.stack(yList, axis = 2)
test2 = test.reshape(test.shape[2], test.shape[1], test.shape[0])

In [69]:
test = np.array([np.arange(8)] * 100)
test2 = np.swapaxes(np.expand_dims(a = test, axis = 2), axis1 = 0, axis2 = 2)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [51]:
np.allclose(distances, wassersteinDists)

True

In [44]:
np.array(wassersteinDists)

array([1.        , 3.16868241, 1.        , 2.27735563, 4.69882935,
       6.35978282, 3.32832636, 3.17528684, 2.        , 6.35165597,
       4.27172399, 4.1430863 , 5.17263576, 3.25543048, 4.06045489,
       4.1305019 , 2.6918518 , 5.        , 6.07503306, 3.57622848,
       3.9319302 , 2.52364117, 3.26045734, 3.84968496, 2.89554205,
       3.33832629, 4.93636408, 4.49380467, 3.86651643, 5.35971353,
       3.71164571, 4.        , 4.50353702, 5.68447095, 4.50769364,
       3.17727277, 4.31643631, 3.15451437, 2.58919977, 2.46577716,
       3.09037864, 6.49772958, 4.5915561 , 3.08677208, 1.        ,
       1.98863491, 4.47545828, 2.52657207, 2.21821329, 6.645546  ,
       3.4182691 , 2.95426494, 4.38018196, 3.        , 3.9142907 ,
       6.05294536, 5.24599213, 1.67578836, 2.90432041, 4.03397428,
       5.3693118 , 3.61789388, 3.86595306, 2.71220836, 1.        ,
       4.49428396, 1.        , 4.26702074, 4.52820191, 3.        ,
       3.12795023, 4.76540598, 3.7940291 , 6.46033906, 5.28328

In [126]:
lenAxis1 = valuesList[0].shape[1]
lenAxis2 = len(valuesList)

probsList2 = list()
valuesList2 = list()
lenList = list()
yList2 = list()

for i in range(len(valuesList)):
    probsList2.append(probsList[i])
    valuesList2.append(valuesList[i])
    yList2.append(yList[i])
    lenList.append(valuesList[i].shape[0])
    
lenMax = max(lenList)
insertionCheckProbs = np.arange(lenMax) < np.array(lenList)[:, None]
insertionCheckProbs = np.swapaxes(insertionCheckProbs, axis1 = 0, axis2 = 1)
# insertionCheckValues = np.concatenate([insertionCheckProbs] * lenAxis1, axis = 1)

probsArray = np.zeros(shape = (lenMax, lenAxis2))
valuesArray = np.zeros(shape = (lenMax, lenAxis1, lenAxis2))

valuesArray[insertionCheckProbs] = np.stack(valuesList, axis = 2)
probsArray[insertionCheckProbs] = np.stack(probsList, axis = 1)

valuesArray = np.stack(valuesList, axis = 2)
probsArray = np.stack(probsList, axis = 1)
yArray = np.stack(yList, axis = 2)

distances = np.sum(np.abs(valuesArray - yArray)**2, axis = 1)

test = np.sum(probsArray * distances, axis = 0)**(1/2)

IndexError: boolean index did not match indexed array along dimension 1; dimension is 8 but corresponding boolean dimension is 100

In [197]:
lenList = list()

for i in range(len(valuesList)):
    lenList.append(valuesList[i].shape[0])
lenMax = max(lenList)

insertionCheck = np.arange(lenMax) < np.array(lenList)[:, None, None]
insertionCheck2 = np.repeat(a = insertionCheck, repeats = 8, axis = 1)

# insertionCheckProbs = np.swapaxes(insertionCheckProbs, axis1 = 0, axis2 = 1)
# insertionCheckValues = np.concatenate([insertionCheckProbs] * lenAxis1, axis = 1)

probsArray = np.zeros(shape = (len(probsList), lenMax))
valuesArray = np.zeros(shape = (len(probsList), valuesList[0].shape[1], lenMax))

# probsArray[insertionCheck] = np.concatenate(probsList, axis = 0)
valuesArray[insertionCheck2] = np.concatenate(valuesList, axis = 0)

# valuesArray[insertionCheckProbs] = np.stack(valuesList, axis = 2)
# probsArray[insertionCheckProbs] = np.stack(probsList, axis = 1)

# valuesArray = np.stack(valuesList, axis = 2)
# probsArray = np.stack(probsList, axis = 1)
# yArray = np.stack(yList, axis = 2)

# distances = np.sum(np.abs(valuesArray - yArray)**2, axis = 1)

# test = np.sum(probsArray * distances, axis = 0)**(1/2)

TypeError: NumPy boolean array indexing assignment requires a 0 or 1-dimensional input, input has 2 dimensions

In [212]:
import itertools
test = list(itertools.zip_longest(*valuesList, fillvalue = 0))
test[0]
itertools.zip_longest?

Init signature: itertools.zip_longest(self, /, *args, **kwargs)
Docstring:     
zip_longest(iter1 [,iter2 [...]], [fillvalue=None]) --> zip_longest object

Return a zip_longest object whose .__next__() method returns a tuple where
the i-th element comes from the i-th iterable argument.  The .__next__()
method continues until the longest iterable in the argument sequence
is exhausted and then it raises StopIteration.  When the shorter iterables
are exhausted, the fillvalue is substituted in their place.  The fillvalue
defaults to None or can be specified by a keyword argument.
Type:           type
Subclasses:     


In [174]:
valuesArray.shape

(100, 8, 9)

In [173]:
valuesList[4].shape

(4, 8)

In [156]:
(np.arange(lenMax) < np.array(lenList)[:, None]).shape

(100, 10)

In [141]:
probsArray

array([[0.01009766, 0.24651637, 0.02263885, 0.20883241, 0.03724715,
        0.0983117 , 0.01659117, 0.24016372, 0.00501884, 0.11458214],
       [0.24778259, 0.14732609, 0.16164974, 0.1079876 , 0.00595314,
        0.0151889 , 0.07955756, 0.00853273, 0.1244222 , 0.10159946],
       [0.1874065 , 0.1481619 , 0.05011163, 0.07397696, 0.06604033,
        0.09255353, 0.04293366, 0.13601729, 0.18094297, 0.02185523],
       [0.02576896, 0.02373454, 0.16745852, 0.13897852, 0.08803924,
        0.1619214 , 0.06923618, 0.09773776, 0.13964836, 0.08747651],
       [0.09522165, 0.07164955, 0.24458031, 0.08281456, 0.00843574,
        0.06709888, 0.15791992, 0.24180813, 0.0090158 , 0.02145545],
       [0.02008905, 0.17642716, 0.16596693, 0.02654114, 0.01316914,
        0.17595823, 0.04821761, 0.13583665, 0.09388884, 0.14390525],
       [0.17741677, 0.13820225, 0.05235513, 0.01912789, 0.1053217 ,
        0.09786278, 0.1133941 , 0.00784073, 0.16389146, 0.12458717],
       [0.15556941, 0.01436101, 0.0958918

In [122]:
valuesArray.shape

(10, 8, 100)

In [123]:
insertionCheckProbs.shape

(10, 100)

In [129]:
insertionCheckProbs.shape

(10, 100)

In [127]:
valuesArray.shape

(10, 8, 100)

## One dimensional

In [9]:
valuesList = [np.ravel([randint(low = 0, high = 10, size = 1) for i in range(10)]) for j in range(10000)]
probsList = [np.ravel(np.array([uniform(low = 0, high = 1, size = 1) for i in range(10)])) for j in range(10000)]
probsList = [probs / probs.sum() for probs in probsList]

yList = [np.ravel([randint(low = 0, high = 10, size = 1) for i in range(1)]) for j in range(10000)]

In [22]:
wassersteinDists = list()
    
for i in range(len(valuesList)):
    y = yList[i]
    values = valuesList[i]
    probs = probsList[i]
    
    values = values.reshape(-1, 1)
    y = y.reshape(-1, 1)
    
    wassersteinDists.append(np.sum(probs * np.sum(np.abs(values - y)**2, axis = 1))**(1/2))

In [10]:
%%timeit

valuesArray = np.stack(valuesList, axis = 2)
yArray = np.stack(yList, axis = 2)
probsArray = np.stack(probsList, axis = 1)

distances = np.sum(np.abs(valuesArray - yArray)**2, axis = 1)

test = np.sum(probsArray * distances, axis = 0)**(1/2)

45.1 ms ± 623 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


array([ 8.93145305,  7.05747364,  6.48070443,  6.15345144,  8.47698363,
        8.24493505,  5.74488632,  7.82924342,  8.32609941,  7.10131074,
       10.9406129 ,  8.065638  ,  7.36269968,  8.18294643,  6.64663468,
        7.06436941,  9.49619426,  5.93776717,  9.04891493,  6.3582101 ,
        7.00127067,  6.39544313,  7.91396806,  7.36734365,  6.0999823 ,
        9.90770018,  6.01336787,  6.58362521,  6.9341847 ,  9.79406372,
        7.45145959,  8.6216385 ,  7.25382353,  8.09243922,  8.77022475,
        7.26595275,  8.44040013,  7.5716263 ,  9.90639136,  8.36780627,
        7.45561185,  6.47011865,  6.96234616,  8.02718346,  6.79192255,
        9.62410203,  9.40918537,  8.44175407,  7.47013667,  9.99245647,
        8.96108021,  6.51480403,  6.43502126,  7.04753586,  9.24312877,
        7.04072998,  6.30885871,  7.04557355,  6.84027153,  7.85331342,
        6.59377425, 11.13239999,  7.8855719 ,  9.30227178,  9.26060035,
        6.81829361,  6.79912063,  7.8972269 ,  8.8083585 ,  9.70

In [134]:
wassersteinDists

[8.931453050666557,
 7.0574736407974505,
 6.480704427711937,
 6.153451435266372,
 8.476983632846846,
 8.24493504566945,
 5.74488631984587,
 7.829243421874024,
 8.326099406071085,
 7.101310744010564,
 10.940612896892505,
 8.065637997437424,
 7.362699683386023,
 8.182946432382343,
 6.6466346794379625,
 7.06436940548583,
 9.496194264726078,
 5.937767171903744,
 9.048914927232289,
 6.3582101018600605,
 7.00127066609408,
 6.395443131559743,
 7.913968055479238,
 7.3673436543813375,
 6.099982296312708,
 9.9077001782744,
 6.013367865206719,
 6.583625206795354,
 6.934184699123185,
 9.794063719326253,
 7.451459592396499,
 8.621638499426497,
 7.25382352919832,
 8.092439218750528,
 8.770224751691122,
 7.265952752360882,
 8.44040012920247,
 7.571626295396472,
 9.906391361642418,
 8.36780626916971,
 7.455611848301641,
 6.470118650809714,
 6.9623461566178415,
 8.027183464016158,
 6.79192255406364,
 9.624102032536126,
 9.409185370638838,
 8.441754073308255,
 7.47013667362885,
 9.992456465856455,
 8.96

# SCANN

In [ ]:
import numpy as np
import h5py
import os
import requests
import tempfile
import time

import scann

In [ ]:
with tempfile.TemporaryDirectory() as tmp:
    response = requests.get("http://ann-benchmarks.com/glove-100-angular.hdf5")
    loc = os.path.join(tmp, "glove.hdf5")
    with open(loc, 'wb') as f:
        f.write(response.content)
    
    glove_h5py = h5py.File(loc, "r")

In [ ]:
list(glove_h5py.keys())

In [ ]:
dataset = glove_h5py['train']
queries = glove_h5py['test']
print(dataset.shape)
print(queries.shape)

In [ ]:
normalized_dataset = dataset / np.linalg.norm(dataset, axis=1)[:, np.newaxis]
# configure ScaNN as a tree - asymmetric hash hybrid with reordering
# anisotropic quantization as described in the paper; see README

# use scann.scann_ops.build() to instead create a TensorFlow-compatible searcher
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 100, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [ ]:
def compute_recall(neighbors, true_neighbors):
    total = 0
    for gt_row, row in zip(true_neighbors, neighbors):
        total += np.intersect1d(gt_row, row).shape[0]
    return total / true_neighbors.size

In [ ]:
# this will search the top 100 of the 2000 leaves, and compute
# the exact dot products of the top 100 candidates from asymmetric
# hashing to get the final top 10 candidates.
start = time.time()
neighbors, distances = searcher.search_batched(queries)
end = time.time()

# we are given top 100 neighbors in the ground truth, so select top 10
print("Recall:", compute_recall(neighbors, glove_h5py['neighbors'][:, :100]))
print("Time:", end - start)

In [ ]:
start = time.time()
neighbors, distances = searcher.search_batched(queries, leaves_to_search=500)
end = time.time()

print("Recall:", compute_recall(neighbors, glove_h5py['neighbors'][:, :100]))
print("Time:", end - start)

In [ ]:
neighbors

# Multivariate Preds Test

In [2]:
a = np.array([randint(low = 0, high = 10000, size = 4) for i in range(1000)])
# a = np.concatenate([a, np.array([np.arange(2)] * 10)], axis = 0)
a.shape

(1000, 4)

## Clustering

In [3]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import numpy as np
from faiss import Kmeans

def get_even_clusters(X, cluster_size):
    n_clusters = int(np.ceil(len(X)/cluster_size))
    kmeans = Kmeans(d = X.shape[1], k = n_clusters)
    
    start = time.time()
    kmeans.train(X)
    print(time.time() - start)
    
    centers = kmeans.centroids
    centers = centers.reshape(-1, 1, X.shape[-1]).repeat(cluster_size, 1).reshape(-1, X.shape[-1])
    
    start = time.time()
    distance_matrix = cdist(X, centers)
    print(time.time() - start)
    
    start = time.time()
    clusters = linear_sum_assignment(distance_matrix)[1]//cluster_size
    print(time.time() - start)
    
    return centers

In [5]:
X = a.astype(np.float32)
cluster_size = 100

import time
import ipdb

res = get_even_clusters(X, cluster_size)


5.873585939407349
0.007401227951049805
0.10688900947570801


In [ ]:
Counter(res)

In [ ]:
X = a.astype(np.float32)
kmeans = KMeans(n_clusters)

kmeans.fit(X)

In [ ]:
centers = kmeans.cluster_centers_

In [ ]:
centers = centers.reshape(-1, 1, X.shape[-1]).repeat(cluster_size, 1).reshape(-1, X.shape[-1])
distance_matrix = cdist(X, centers)

In [ ]:
clusters = linear_sum_assignment(distance_matrix)[1]//cluster_size

In [ ]:
from faiss import Kmeans

In [ ]:
n_init = 10
max_iter = 300
# kmeans = faiss.Kmeans(d = X.shape[1], k = n_clusters, niter = max_iter, nredo = n_init)
kmeans = Kmeans(d = X.shape[1], k = 100)
kmeans.train(X.astype(np.float32))

In [ ]:
res = kmeans.assign(X.astype(np.float32))

In [ ]:
kmeans.centroids

In [ ]:
from collections import Counter
Counter(res[1])

## kNN

In [ ]:
kd = KDTree(data_pts = a)

In [ ]:
start = time.time()
dist, idx = kd.query(a, k = 2000)
print(time.time() - start)

In [ ]:
idx

In [ ]:
nn = NearestNeighbors(algorithm = 'kd_tree')

In [ ]:
nn.fit(a)

In [ ]:
start = time.time()
idx2 = nn.kneighbors(a, n_neighbors = 10000, return_distance = False)
print(time.time() - start)

In [ ]:
np.array_equal(idx, idx2)

In [ ]:
def bad_append(new_item, a_list=[]):
    a_list.append(new_item)
    return a_list

In [ ]:
bad_append('one', a_list = [1, 2])
print(bad_append('one', a_list = [1, 2]))

In [ ]:
def good_append(new_item, a_list=None):
    if a_list is None:
        a_list = []
    a_list.append(new_item)
    return a_list

In [ ]:
print(good_append('one'))

In [ ]:
import numpy as np
import pandas as pd
import os
from random import sample

from lightgbm import LGBMRegressor
from dddex.levelSetKDEx import LevelSetKDEx, binSizeCV
from dddex.utils import groupedTimeSeriesSplit

In [ ]:
path = ('/home/kagu/yaz_trainingData/dataYaz.csv')
data = pd.read_csv(path)

In [ ]:
dataTrain = data[data['label'] == 'train'].reset_index(drop = True)
dataTest = data[data['label'] == 'test']

XTrain = dataTrain.drop(['dayIndex', 'label', 'id', 'demand'], axis = 1)
XTest = dataTest.drop(['dayIndex', 'label', 'id', 'demand'], axis = 1)

yTrain = dataTrain['demand']
yTest = dataTest['demand']

In [ ]:
LGBM = LGBMRegressor(n_jobs = 1)
LGBM.fit(X = XTrain, y = yTrain)

In [ ]:
cvFolds = groupedTimeSeriesSplit(data = dataTrain, kFolds = 2, testLength = 28, groupFeature = 'id', timeFeature = 'dayIndex')

In [ ]:
res = binSizeCV(estimator = LGBM,
                binSizeGrid = [100, 200],
                cv = cvFolds,
                LSF_type = 'LSF')

In [ ]:
res.fit(X = XTrain, y = yTrain)

# Cross Validation - Timing

In [1]:
import numpy as np
import pandas as pd

from dddex.crossValidation import *
from dddex.levelSetKDEx_univariate import *
from dddex.loadData import loadDataYaz

from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

import time

In [2]:
data, XTrain, yTrain, XTest, yTest = loadDataYaz()

In [3]:
LGBM = LGBMRegressor(n_jobs = 1)
LSKDEx = LevelSetKDEx(estimator = LGBM)

In [4]:
paramGridLSx = {'binSize': [20, 50, 100, 150, 200, 250, 400, 600, 800, 1000],
                'weightsByDistance': [False, True]}

paramGridEstimator = {'max_depth': [4],
                      'n_estimators': [100]}

In [5]:
dataTrain = data[data.label == 'train']

cvFolds = groupedTimeSeriesSplit(data = dataTrain, 
                                 kFolds = 5, 
                                 testLength = 28, 
                                 groupFeature = 'id', 
                                 timeFeature = 'dayIndex')

In [7]:
%%timeit 

CV = CrossValidationLSx_combined(estimatorLSx = LSKDEx,
                                 cvFolds = cvFolds,
                                 parameterGridLSx = paramGridLSx,
                                 parameterGridEstimator = paramGridEstimator,
                                 probs = np.arange(0.01, 0.99, 0.01),
                                 refitPerProb = False,
                                 n_jobs = len(cvFolds))

CV.fit(XTrain, yTrain)

3.61 s ± 337 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
start = time.time()

CV = GridSearchCV(estimator = LGBM,
                  cv = cvFolds,
                  param_grid = paramGridEstimator,
                  scoring = 'neg_mean_squared_error',
                  refit = False,
                  n_jobs = len(cvFolds),
                  return_train_score = True,
                  verbose = 0)

CV.fit(XTrain, yTrain)

print(time.time() - start)

0.3275916576385498


In [11]:
%%timeit

CV = QuantileCrossValidation(quantileEstimator = LSKDEx,
                             cvFolds = cvFolds,
                             parameterGrid = paramGridLSx,
                             probs = np.arange(0.01, 0.99, 0.01),
                             refitPerProb = False,
                             n_jobs = len(cvFolds))

CV.fit(XTrain, yTrain)

4.17 s ± 71.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit

CV = DensityCrossValidation(estimator = LSKDEx,
                            cvFolds = cvFolds,
                            parameterGrid = paramGridLSx,
                            p = 1,
                            n_jobs = len(cvFolds))

CV.fit(XTrain, yTrain)

5.22 s ± 54.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Kernel Matrix Inversion

In [1]:
import numpy as np
import pandas as pd

from dddex.crossValidation import *
from dddex.levelSetKDEx_univariate import *
from dddex.loadData import loadDataYaz

from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

import time
import ipdb

In [42]:
data, XTrain, yTrain, XTest, yTest = loadDataYaz()

In [281]:
LGBM = LGBMRegressor(n_jobs = 1)
LGBM.fit(XTrain, yTrain)

LSKDEx = LevelSetKDEx(estimator = LGBM, 
                      binSize = 100)
LSKDEx.fit(XTrain, yTrain)

In [236]:
yPred = LGBM.predict(XTrain)[0:10]
binPerPred = np.searchsorted(a = LSKDEx.lowerBoundPerBin, v = yPred, side = 'right') - 1

In [237]:
binVectors = [(binPerPred == i).reshape(-1, 1) * 1 for i in range(len(LSKDEx.lowerBoundPerBin))]
binVectorsToSlice = [np.where(binVector)[0] for binVector in binVectors]

In [238]:
sigma = 0.5
sigmaI = np.identity(yPred.shape[0]) * sigma

In [239]:
kernelMatrixList = [np.matmul(binVector, binVector.T) for binVector in binVectors]

In [240]:
kernelMatrix = kernelMatrixList[0] + sigmaI

In [241]:
sigmaInverse = np.identity(yPred.shape[0]) * sigma**(-1)

comp = np.sum(sigmaInverse[:, binVectorsToSlice[0]], axis = 1).reshape(-1, 1)

correctionMatrix = np.matmul(comp, comp.T)
denominator = 1 + np.sum(comp[binVectorsToSlice[0]])
correctionMatrix = correctionMatrix / denominator

kernelInvers = sigmaInverse - correctionMatrix

In [242]:
identity = np.matmul(kernelMatrix, kernelInvers)

In [243]:
np.allclose(np.identity(yPred.shape[0]), identity)

True

## GLS Approach

In [3]:
data, XTrain, yTrain, XTest, yTest = loadDataYaz()

In [4]:
n = 30

XTrainDup = np.concatenate([XTrain] * n, axis = 0)
yTrainDup = np.concatenate([yTrain] * n, axis = 0)

LGBM = LGBMRegressor(n_jobs = 1)
LGBM.fit(XTrain, yTrain)

LSKDEx = LevelSetKDEx(estimator = LGBM, 
                      binSize = 100 * n)
LSKDEx.fit(XTrainDup, yTrainDup)

In [5]:
yPred = LGBM.predict(XTrainDup)
binPerPred = np.searchsorted(a = LSKDEx.lowerBoundPerBin, v = yPred, side = 'right') - 1

binVectors = [(binPerPred == i).reshape(-1, 1) * 1 for i in range(len(LSKDEx.lowerBoundPerBin))]
binVectorsToSlice = [np.where(binVector)[0] for binVector in binVectors]

In [23]:
c = np.array([np.arange(binVectors[0].shape[0])] * 10000).T
sigma = 0.5

In [24]:
def getNewSolution(u, x, y):
    
    if len(u.shape) == 1:
        u.reshape(-1, 1)
        
    uSlice = np.where(u == 1)[0]
    
    ux = np.sum(x[uSlice, :], axis = 0, keepdims = True)
    xNew = x - np.matmul(y, ux) / (1 + np.sum(y[uSlice]))
    
    return xNew

In [25]:
def solveGLS_initial(sigma, u, c):
    
    if len(u.shape) == 1:
        u = u.reshape(-1, 1)
        
    if len(c.shape) == 1:
        c = c.reshape(-1, 1)
    
    x = c * sigma**(-1)
    y = u * sigma**(-1)
    
    return getNewSolution(u = u, x = x, y = y)

In [26]:
start = time.time()

# CALCULATING ALL NECESSARY y's

yDict = dict()

# Loop for A_k^-1 y = u_k
for k in range(len(binVectors)):
        
    # Loop for A_j^-1 y = u_k
    for j in range(k):

        if j == 0:
            yDict[(0, k)] = solveGLS_initial(sigma = sigma, u = binVectors[0], c = binVectors[k])

        else:
            yDict[j, k] = getNewSolution(u = binVectors[j], x = yDict[(j - 1, k)], y = yDict[(j - 1, j)])   
    
    #---
    
    if k == 0:
        x = solveGLS_initial(sigma = sigma, u = binVectors[0], c = c)
    else:
        x = getNewSolution(u = binVectors[k], x = x, y = yDict[(k - 1, k)])
        
print(time.time() - start)

132.29863858222961


In [ ]:
# A0 = np.identity(yPred.shape[0]) * sigma

# kernelMatrixList = [np.matmul(binVector, binVector.T) for binVector in binVectors]
# kernelMatrix = A0 + sum(kernelMatrixList)

# res = np.matmul(kernelMatrix, x)

In [376]:
def solveGLS_initial2(sigma, u, c):
    
    if len(u.shape) == 1:
        u = u.reshape(-1, 1)
        
    if len(c.shape) == 1:
        c = c.reshape(-1, 1)
    
    #---
    
    uSlice = np.where(u == 1)[0]
    
    x = c * sigma**(-1)
    y = u * sigma**(-1)
    
    ux = np.sum(x[uSlice, :], axis = 0, keepdims = True)
    xNew = x - np.matmul(y, ux) / (1 + np.sum(y[uSlice]))
    
    return xNew

In [230]:
# MULTI-DIMENSIONAL

u = np.array([1, 0, 1]).reshape(-1, 1)
c = np.array([[1, 1],
              [2, 2],
              [3, 3]])
sigma = 0.5

x = solveGLS_initial(sigma = sigma, u = u, c = c)

np.matmul(np.identity(u.shape[0]) * sigma + np.matmul(u, u.T), x)

array([[1., 1.],
       [2., 2.],
       [3., 3.]])

In [231]:
# ONE-DIMENSIONAL

u = np.array([1, 0, 1]).reshape(-1, 1)
c = np.array([1, 2, 3]).reshape(-1, 1)
sigma = 0.5

x = solveGLS_initial(sigma = sigma, u = u, c = c)

np.matmul(np.identity(u.shape[0]) * sigma + np.matmul(u, u.T), x)

array([[1.],
       [2.],
       [3.]])

In [258]:
sigma = 0.5
c = np.arange(10).reshape(-1, 1)
u = binVectors[0]

x = solveGLS_initial(sigma = sigma, u = binVectors[0], c = c)
y = solveGLS_initial(sigma = sigma, u = binVectors[0], c = binVectors[1])

    
u = binVectors[1]

if len(u.shape) == 1:
    u = u.reshape(-1, 1)

#---

uSlice = np.where(u == 1)[0]


    
    

In [259]:
u0 = binVectors[0]
u1 = binVectors[1]

np.matmul(np.identity(u0.shape[0]) * sigma + np.matmul(u0, u0.T) + np.matmul(u1, u1.T), xNew)

array([[-7.10542736e-15],
       [ 1.00000000e+00],
       [ 2.00000000e+00],
       [ 3.00000000e+00],
       [ 4.00000000e+00],
       [ 5.00000000e+00],
       [ 6.00000000e+00],
       [ 7.00000000e+00],
       [ 8.00000000e+00],
       [ 9.00000000e+00]])

In [ ]:
# def solveGLS(sigma, binVectors, c):
    
#     for k in np.flip(np.arange(len(binVectors))):
        
#         if k > 0:
#             x0 = solveGLS(sigma = sigma, binVectors = binVectors[0:k], c = c)
        
#         else:    

## Matrix Approach

In [325]:
n = 4

XTrainDup = np.concatenate([XTrain] * n, axis = 0)
yTrainDup = np.concatenate([yTrain] * n, axis = 0)

LGBM = LGBMRegressor(n_jobs = 1)
LGBM.fit(XTrain, yTrain)

LSKDEx = LevelSetKDEx(estimator = LGBM, 
                      binSize = 100 * n)
LSKDEx.fit(XTrainDup, yTrainDup)

In [326]:
yPred = LGBM.predict(XTrainDup)
binPerPred = np.searchsorted(a = LSKDEx.lowerBoundPerBin, v = yPred, side = 'right') - 1

binVectors = [(binPerPred == i).reshape(-1, 1) * 1 for i in range(len(LSKDEx.lowerBoundPerBin))]
binVectorsToSlice = [np.where(binVector)[0] for binVector in binVectors]

sigma = 0.5
A0 = np.identity(yPred.shape[0]) * sigma

In [327]:
B0 = np.identity(yPred.shape[0]) * sigma**(-1)

comp = np.sum(B0[:, binVectorsToSlice[0]], axis = 1).reshape(-1, 1)

correctionMatrix = np.matmul(comp, comp.T)
denominator = 1 + np.sum(comp[binVectorsToSlice[0]])
correctionMatrix = correctionMatrix / denominator

B = B0 - correctionMatrix

In [328]:
start = time.time()

for i in range(len(binVectors))[1:]:

    comp = np.sum(B[:, binVectorsToSlice[i]], axis = 1).reshape(-1, 1)

    correctionMatrix = np.matmul(comp, comp.T)
    denominator = 1 + np.sum(comp[binVectorsToSlice[i]])
    correctionMatrix = correctionMatrix / denominator

    B = B - correctionMatrix
    
print(time.time() - start)

31.28031325340271


In [309]:
kernelMatrixList = [np.matmul(binVector, binVector.T) for binVector in binVectors]
kernelMatrix = A0 + sum(kernelMatrixList)

In [310]:
identity = np.matmul(kernelMatrix, B)

In [311]:
np.allclose(np.identity(yPred.shape[0]), identity)

True

In [290]:
kernelMatrix.shape

(10, 10)